In [2]:
import os
import json
import pandas as pd
import pysrt
import re

In [3]:
def compute_iou(interval_1, interval_2):
    start_i, end_i = float(interval_1[0]), float(interval_1[1])
    start, end = float(interval_2[0]), float(interval_2[1])
    intersection = max(0, min(end, end_i) - max(start, start_i))
    union = min(max(end, end_i) - min(start, start_i), end-start + end_i-start_i)
    iou = float(intersection) / (union + 1e-8)
    return iou

In [4]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    #print(data)
    return data

BASELINE Evaluation

In [5]:
val = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val.json')
test = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test.json')

In [6]:
folders = ['/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_decompose_query_exp/baseline', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_decompose_query_exp/visual_med', 
           #'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_decompose_query_exp/visual_nonmed', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_decompose_query_exp/visual_med_with_audio']

In [7]:
test_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_test_queries.txt"
val_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_val_queries.txt"
test_med_query_f_metamap_list, val_med_query_f_metamap_list = [], []
with open(test_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        if line.startswith(" ") or line.startswith("-"):
            line = line.lstrip("-").lstrip(" ")
        test_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
f.close()
with open(val_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        # if line.startswith(" ") or line.startswith("-"):
        #     line = line.lstrip("-").lstrip(" ")
        val_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
        #val_med_query_f_metamap_list.append(line.strip(" ").strip("\n").replace("  ", " "))
f.close()
print(len(test_med_query_f_metamap_list), len(val_med_query_f_metamap_list))

246 569


In [8]:
test_queries = {'med':[], 'nonmed': []}
val_queries = {'med':[], 'nonmed': []}
med_total, nonmed_total = 0, 0
for key, value in test[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in test_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        test_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        test_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    
print(med_total, nonmed_total)
med_total, nonmed_total = 0, 0
for key, value in val[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in val_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        val_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        val_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])

print(med_total, nonmed_total)

246 7372
569 14675


In [9]:
folders = ['baseline', 'visual_med', 'visual_nonmed', 'visual_med_with_audio']

In [16]:
root = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_decompose_query_exp"

In [11]:
vid_duration_json = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/video_duration.json"
video_duration_dict = load_jsonl(vid_duration_json)[0]

In [12]:
clip_pred_med = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/five_labeled_pred_med_from_gt_vid_dict.json')

In [31]:
select_option = "union"
#select_option = "intersect"

In [32]:
for folder in folders:
    vid_json_folder = os.path.join(root, folder)

    print(f"{folder}\n")

    med_test_result_json_path = f"{vid_json_folder}/medical_test/all_data_test_moment_retrieval_BEST_with_audio.json"
    nonmed_test_result_json_path = f"{vid_json_folder}/nonmedical_test/all_data_test_moment_retrieval_BEST_with_audio.json"
    med_val_result_json_path = f"{vid_json_folder}/medical_val/all_data_val_moment_retrieval_BEST_with_audio.json"
    nonmed_val_result_json_path = f"{vid_json_folder}/nonmedical_val/all_data_val_moment_retrieval_BEST_with_audio.json"

    med_test_result_org_json_path = f"{vid_json_folder}/medical_test/all_data_test.json"
    nonmed_test_result_org_json_path = f"{vid_json_folder}/nonmedical_test/all_data_test.json"
    med_val_result_json_org_path = f"{vid_json_folder}/medical_val/all_data_val.json"
    nonmed_val_result_json_org_path = f"{vid_json_folder}/nonmedical_val/all_data_val.json"

    val = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val.json')
    test = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test.json')

    med_test_result = load_result_json(med_test_result_json_path)
    nonmed_test_result = load_result_json(nonmed_test_result_json_path)
    med_val_result = load_result_json(med_val_result_json_path)
    nonmed_val_result = load_result_json(nonmed_val_result_json_path)

    med_test_org = load_result_json(med_test_result_org_json_path)
    nonmed_test_org = load_result_json(nonmed_test_result_org_json_path)
    med_val_org = load_result_json(med_val_result_json_org_path)
    nonmed_val_org = load_result_json(nonmed_val_result_json_org_path)

    new_med_val_result = {}
    #print(med_val_org)
    for key in med_val_org:
        
        # print(key, med_val_org[key])
        # print(vid)
        value = med_val_org[key]
        vid = next(iter(value))
        if value['original_qa'] not in new_med_val_result:
            new_dict = {}
            new_dict[vid] = {}
            new_dict[vid]['bounds'] = []
            new_dict[vid]['relevant'] = True
            new_dict[vid]['clip'] = True
            new_dict[vid]['v_duration'] = value[vid]['v_duration']
            new_dict[vid]['steps'] = []
            new_dict[vid]['all_bounds'] = [med_val_result[key][vid]['bounds']]
            new_med_val_result[value['original_qa']] = new_dict
        else:
            if vid not in new_med_val_result[value['original_qa']]:
                new_dict = {}
                new_dict[vid] = {}
                new_dict[vid]['bounds'] = []
                new_dict[vid]['relevant'] = True
                new_dict[vid]['clip'] = True
                new_dict[vid]['v_duration'] = value[vid]['v_duration']
                new_dict[vid]['steps'] = []
                new_dict[vid]['all_bounds'] = [med_val_result[key][vid]['bounds']]
                new_med_val_result[value['original_qa']] = new_dict
            else:
                new_med_val_result[value['original_qa']][vid]['all_bounds'].append(med_val_result[key][vid]['bounds']) 

    new_nonmed_val_result = {}
    for key in nonmed_val_org:
        
        value = nonmed_val_org[key]
        vid = next(iter(value))
        if value['original_qa'] not in new_nonmed_val_result:
            new_dict = {}
            new_dict[vid] = {}
            new_dict[vid]['bounds'] = []
            new_dict[vid]['relevant'] = True
            new_dict[vid]['clip'] = True
            new_dict[vid]['v_duration'] = value[vid]['v_duration']
            new_dict[vid]['steps'] = []
            new_dict[vid]['all_bounds'] = [nonmed_val_result[key][vid]['bounds']]
            new_nonmed_val_result[value['original_qa']] = new_dict
        else:
            if vid not in new_nonmed_val_result[value['original_qa']]:
                new_dict = {}
                new_dict[vid] = {}
                new_dict[vid]['bounds'] = []
                new_dict[vid]['relevant'] = True
                new_dict[vid]['clip'] = True
                new_dict[vid]['v_duration'] = value[vid]['v_duration']
                new_dict[vid]['steps'] = []
                new_dict[vid]['all_bounds'] = [nonmed_val_result[key][vid]['bounds']]
                new_nonmed_val_result[value['original_qa']] = new_dict
            else:
                new_nonmed_val_result[value['original_qa']][vid]['all_bounds'].append(nonmed_val_result[key][vid]['bounds']) 

    new_med_test_result = {}
    for key in med_test_org:
        value = med_test_org[key]
        vid = next(iter(value))
        if value['original_qa'] not in new_med_test_result:
            new_dict = {}
            new_dict[vid] = {}
            new_dict[vid]['bounds'] = []
            new_dict[vid]['relevant'] = True
            new_dict[vid]['clip'] = True
            new_dict[vid]['v_duration'] = value[vid]['v_duration']
            new_dict[vid]['steps'] = []
            new_dict[vid]['all_bounds'] = [med_test_result[key][vid]['bounds']]
            new_med_test_result[value['original_qa']] = new_dict
        else:
            if vid not in new_med_test_result[value['original_qa']]:
                new_dict = {}
                new_dict[vid] = {}
                new_dict[vid]['bounds'] = []
                new_dict[vid]['relevant'] = True
                new_dict[vid]['clip'] = True
                new_dict[vid]['v_duration'] = value[vid]['v_duration']
                new_dict[vid]['steps'] = []
                new_dict[vid]['all_bounds'] = [med_test_result[key][vid]['bounds']]
                new_med_test_result[value['original_qa']] = new_dict
            else:
                new_med_test_result[value['original_qa']][vid]['all_bounds'].append(med_test_result[key][vid]['bounds']) 
    new_nonmed_test_result = {}
    for key in nonmed_test_org:
        value = nonmed_test_org[key]
        vid = next(iter(value))
        if value['original_qa'] not in new_nonmed_test_result:
            new_dict = {}
            new_dict[vid] = {}
            new_dict[vid]['bounds'] = []
            new_dict[vid]['relevant'] = True
            new_dict[vid]['clip'] = True
            new_dict[vid]['v_duration'] = value[vid]['v_duration']
            new_dict[vid]['steps'] = []
            new_dict[vid]['all_bounds'] = [nonmed_test_result[key][vid]['bounds']]
            new_nonmed_test_result[value['original_qa']] = new_dict
        else:
            if vid not in new_nonmed_test_result[value['original_qa']]:
                new_dict = {}
                new_dict[vid] = {}
                new_dict[vid]['bounds'] = []
                new_dict[vid]['relevant'] = True
                new_dict[vid]['clip'] = True
                new_dict[vid]['v_duration'] = value[vid]['v_duration']
                new_dict[vid]['steps'] = []
                new_dict[vid]['all_bounds'] = [nonmed_test_result[key][vid]['bounds']]
                new_nonmed_test_result[value['original_qa']] = new_dict
            else:
                new_nonmed_test_result[value['original_qa']][vid]['all_bounds'].append(nonmed_test_result[key][vid]['bounds']) 

    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, cnt = 0, 0, 0, 0


    for key, value in test[0].items():
        if key in new_med_test_result:
            #print(key)
            vid = next(iter(value))
            intersect_bound = []
            for bound in new_med_test_result[key][vid]['all_bounds']:
                if len(intersect_bound) == 0:
                    intersect_bound = bound
                else:
                    if select_option == "intersect":
                        start = max(intersect_bound[0], bound[0])
                        end = min(intersect_bound[1], bound[1])
                    elif select_option == "union":
                        start = min(intersect_bound[0], bound[0])
                        end = max(intersect_bound[1], bound[1])
                    intersect_bound = [start, end]
            iou = compute_iou(value[vid]['bounds'], intersect_bound)
            cnt += 1
            if iou >= 0.3:
                med_iou_thresh_3 += 1 
            if iou >= 0.5:
                med_iou_thresh_5 += 1 
            if iou >= 0.7:
                med_iou_thresh_7 += 1 
    print(f"Med TEST IoU=0.3: {med_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in test[0].items():
        if key in new_nonmed_test_result:
            vid = next(iter(value))
            intersect_bound = []
            for bound in new_nonmed_test_result[key][vid]['all_bounds']:
                if len(intersect_bound) == 0:
                    intersect_bound = bound
                else:
                    if select_option == "intersect":
                        start = max(intersect_bound[0], bound[0])
                        end = min(intersect_bound[1], bound[1])
                    elif select_option == "union":
                        start = min(intersect_bound[0], bound[0])
                        end = max(intersect_bound[1], bound[1])
                    intersect_bound = [start, end]
            iou = compute_iou(value[vid]['bounds'], intersect_bound)
            cnt += 1
            if iou >= 0.3:
                nonmed_iou_thresh_3 += 1 
            if iou >= 0.5:
                nonmed_iou_thresh_5 += 1 
            if iou >= 0.7:
                nonmed_iou_thresh_7 += 1 
    print(f"\nNonmed TEST IoU=0.3: {nonmed_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")


    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in val[0].items():
        if key in new_med_val_result:
            vid = next(iter(value))
            intersect_bound = []
            for bound in new_med_val_result[key][vid]['all_bounds']:
                if len(intersect_bound) == 0:
                    intersect_bound = bound
                else:
                    if select_option == "intersect":
                        start = max(intersect_bound[0], bound[0])
                        end = min(intersect_bound[1], bound[1])
                    elif select_option == "union":
                        start = min(intersect_bound[0], bound[0])
                        end = max(intersect_bound[1], bound[1])
                    intersect_bound = [start, end]
            iou = compute_iou(value[vid]['bounds'], intersect_bound)
            cnt += 1
            if iou >= 0.3:
                med_iou_thresh_3 += 1 
            if iou >= 0.5:
                med_iou_thresh_5 += 1 
            if iou >= 0.7:
                med_iou_thresh_7 += 1 
    print(f"\nMed VAL IoU=0.3: {med_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in val[0].items():
        if key in new_nonmed_val_result:
            vid = next(iter(value))
            intersect_bound = []
            for bound in new_nonmed_val_result[key][vid]['all_bounds']:
                if len(intersect_bound) == 0:
                    intersect_bound = bound
                else:
                    if select_option == "intersect":
                        start = max(intersect_bound[0], bound[0])
                        end = min(intersect_bound[1], bound[1])
                    elif select_option == "union":
                        start = min(intersect_bound[0], bound[0])
                        end = max(intersect_bound[1], bound[1])
                    intersect_bound = [start, end]
            iou = compute_iou(value[vid]['bounds'], intersect_bound)
            cnt += 1
            if iou >= 0.3:
                nonmed_iou_thresh_3 += 1 
            if iou >= 0.5:
                nonmed_iou_thresh_5 += 1 
            if iou >= 0.7:
                nonmed_iou_thresh_7 += 1 
    print(f"\nNonmed VAL IoU=0.3: {nonmed_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    print("\n\n")

baseline

Med TEST IoU=0.3: 5.5
IoU=0.5: 3.0
IoU=0.7: 2.0
total queries:200

Nonmed TEST IoU=0.3: 8.0
IoU=0.5: 5.0
IoU=0.7: 2.5
total queries:200

Med VAL IoU=0.3: 9.5
IoU=0.5: 4.5
IoU=0.7: 1.5
total queries:200

Nonmed VAL IoU=0.3: 15.0
IoU=0.5: 4.0
IoU=0.7: 2.5
total queries:200



visual_med

Med TEST IoU=0.3: 10.5
IoU=0.5: 4.0
IoU=0.7: 1.5
total queries:200

Nonmed TEST IoU=0.3: 12.5
IoU=0.5: 4.5
IoU=0.7: 1.0
total queries:200

Med VAL IoU=0.3: 11.5
IoU=0.5: 5.0
IoU=0.7: 1.5
total queries:200

Nonmed VAL IoU=0.3: 8.0
IoU=0.5: 3.5000000000000004
IoU=0.7: 1.5
total queries:200



visual_nonmed

Med TEST IoU=0.3: 7.000000000000001
IoU=0.5: 4.0
IoU=0.7: 1.0
total queries:200

Nonmed TEST IoU=0.3: 9.0
IoU=0.5: 3.0
IoU=0.7: 0.0
total queries:200

Med VAL IoU=0.3: 6.5
IoU=0.5: 2.5
IoU=0.7: 1.0
total queries:200

Nonmed VAL IoU=0.3: 4.0
IoU=0.5: 1.5
IoU=0.7: 0.5
total queries:200



visual_med_with_audio

Med TEST IoU=0.3: 9.5
IoU=0.5: 4.0
IoU=0.7: 1.0
total queries:200

Nonmed TEST IoU=0.